# Selección de características en modelos predictivos

Este proyecto es para IA de Moisés Calzado Cobo y Antonio Germán Márquez Trujillo

Estos archivos contienen los algoritmos implementados, pero no los importamos ya que en esta libreta los implementamos explicando su funcionamiento


In [2]:
import pandas as pandas
import time
import numpy as np
import matplotlib.pyplot as plt
import multiprocessing as mp
from sklearn.model_selection import cross_val_score
from sklearn import tree
from funcy import join
from sklearn import preprocessing
pandas.set_option('max_colwidth', 800)

In [3]:
#datos = pandas.read_csv("datos/titanic.csv")
datos = pandas.read_csv("datos/BreastCancerDataset.csv")
datos.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,diagnosis
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


Para representar los datos tenemos las mismas funciones que en las versiones de código sin paralelizar:

In [9]:
def mostrar_datos(datos):
    datos_tabla = [['Mejor Solución', 'Tamaño', 'Rendimiento']]
    claves = datos.keys()
    
    for clave in claves:
        length = len(clave.split(", "))
        datos_tabla = datos_tabla +[[clave, length, datos[clave]]]

    return pandas.DataFrame(datos_tabla)

In [10]:
def pintar_resultados(resultados): 
    plt.rcdefaults()
    fig, ax = plt.subplots()

    # Example data
    people = resultados.keys()
    y_pos = np.arange(len(people))
    performance = resultado_SFFS.values()

    ax.barh(y_pos, performance, align='center')
    ax.set_yticks(y_pos)
    ax.set_yticklabels(people)
    ax.invert_yaxis()
    ax.set_xlabel('Rendimiento')
    ax.set_title('Selección de características')
    plt.show()

Para la versión paralelizada de los algoritmos SFS y SFFS hemos tenido que cambiar la implementación de la función de evaluar soluciones. En esta ocasión recibe el conjunto de soluciones actuales y la nueva variable a añadir, por lo que facilita la ejecución en paralelo recorriendo el conjunto de variables por añadir:

In [11]:
def evaluar_soluciones(datos, solucion_actual, nueva_variable, objetivo, n_exp, cv, clf = tree.DecisionTreeClassifier(), scoring = "balanced_accuracy"):
    variables = solucion_actual[:]
    variables.append(nueva_variable)
    data_frame = pandas.DataFrame(data=datos)
    X = data_frame[variables]
    y = data_frame[objetivo]
    
    scores = np.mean(cross_val_score(clf, X, y, cv=cv, scoring=scoring))
    
    for i in range(n_exp-1):
        new_scores = np.mean(cross_val_score(clf, X, y, cv=cv, scoring=scoring) )
        scores = scores + new_scores
    
    scores = scores/n_exp
    
    diccionario_resultado = {}
    diccionario_resultado[nueva_variable] = scores
    return diccionario_resultado

Con el fin de paralelizar el algoritmo SFFS, hemos tenido que implementar una función idéntica a la anterior que en esta ocasión elimine una variable del conjunto de soluciones actual. Esto se hace con el fin de paralelizar la ejecución de esta función:

In [12]:
def evaluar_soluciones_eliminando(datos, solucion_actual, variable_a_eliminar, objetivo, n_exp, cv, clf = tree.DecisionTreeClassifier(), scoring = "balanced_accuracy"):
    variables = solucion_actual[:]
    variables.remove(variable_a_eliminar)
    if len(variables) < 1:
        return
    data_frame = pandas.DataFrame(data=datos)
    X = data_frame[variables]
    y = data_frame[objetivo]
    
    scores = np.mean(cross_val_score(clf, X, y, cv=cv, scoring=scoring))
    
    for i in range(n_exp-1):
        new_scores = np.mean(cross_val_score(clf, X, y, cv=cv, scoring=scoring)) 
        scores = scores + new_scores
    
    scores = scores/n_exp
    
    diccionario_resultado = {}
    diccionario_resultado[variable_a_eliminar] = scores
    return diccionario_resultado

Algoritmo SFS paralelizado:

In [37]:
def SFS(datos, respuesta, d = 0):
    start = time.time()

    diccionario_resultado = {}
    columnas = list(datos.columns)
    columnas.remove(respuesta)
    solucion_actual = []

    k = 0
    d = d if d else len(columnas)
    
    while(k<d):
        pool = mp.Pool(mp.cpu_count())
        new_resultados = pool.starmap(evaluar_soluciones, [(datos, solucion_actual, nuevaVariable, respuesta, 15, 10) for nuevaVariable in columnas])
        pool.close()
        resultado = join(new_resultados)
        
        variable_escogida = max(resultado, key=resultado.get)
        solucion_actual.append(variable_escogida)
        columnas.remove(variable_escogida)

        k = k+1
        
        diccionario_resultado[", ".join(solucion_actual)] = resultado[variable_escogida]
    
    done = time.time()
    elapsed = done - start
    print("Tiempo empleado: ", elapsed)
    return diccionario_resultado

In [38]:
resultado_SFS = SFS(datos, "diagnosis", 10)

Tiempo empleado:  15.152360916137695


In [36]:
mostrar_datos(resultado_SFS)

,0,1,2
0,Mejor Solución,Tamaño,Rendimiento
1,worst concave points,1,0.879769
2,"worst concave points, worst radius",2,0.917158
3,"worst concave points, worst radius, mean texture",3,0.939962
4,"worst concave points, worst radius, mean texture, worst smoothness",4,0.947046
5,"worst concave points, worst radius, mean texture, worst smoothness, worst concavity",5,0.947223
6,"worst concave points, worst radius, mean texture, worst smoothness, worst concavity, concavity error",6,0.94517
7,"worst concave points, worst radius, mean texture, worst smoothness, worst concavity, concavity error, mean symmetry",7,0.944835
8,"worst concave points, worst radius, mean texture, worst smoothness, worst concavity, concavity error, mean symmetry, smoothness error",8,0.943214
9,"worst concave points, worst radius, mean texture, worst smoothness, worst concavity, concavity error, mean symmetry, smoothness error, radius error",9,0.943979


Algoritmo SFFS paralelizado:

In [39]:
def SFFS(datos, respuesta):
    start = time.time()
    diccionario_resultado = {}
    soluciones_actual = []
    añadidos = []
    eliminados = []
    columnas = list(datos.columns)
    k = 0
    
    #Compruebo que la variable a predecir no esté en mi conjunto de variables a evaluar
    if respuesta in columnas:
        columnas.remove(respuesta)
    
    while(k<10):
        resultado = []
        score_resultado = 0
        score_resultado_eliminado = 0
        resultado_eliminado = []
        eliminado = ''
        
        #Actualizo el listado de columnas que tengo que evaluar
        columnas_a_evaluar = [x for x in columnas if (x not in añadidos and x not in soluciones_actual)]
                
        if columnas_a_evaluar == []:
            break
        
        #Calculamos el nuevo resultado óptimo
        pool = mp.Pool(mp.cpu_count())
        new_resultados = pool.starmap(evaluar_soluciones, [(datos, soluciones_actual, nuevaVariable, respuesta, 15, 10) for nuevaVariable in columnas_a_evaluar])
        pool.close()
        #Buscamos la variable más óptima
        resultado = join(new_resultados)
        variable_escogida = max(resultado, key=resultado.get)
        score_resultado = resultado[variable_escogida]
        #Añadimos la variable
        soluciones_actual.append(variable_escogida)
        añadidos.append(variable_escogida)
        
        # Pasamos a eliminar variables para comprobar si tengo mejores resultados
        if len(soluciones_actual) > 1:
            variables_a_eliminar = [x for x in soluciones_actual if x not in eliminados]

            pool = mp.Pool(2)
            new_resultados = pool.starmap(evaluar_soluciones_eliminando, [(datos, soluciones_actual, variable_a_eliminar, respuesta, 15, 10) for variable_a_eliminar in variables_a_eliminar])
            pool.close()
            resultado = join(new_resultados)
            variable_a_eliminar = max(resultado, key=resultado.get)
            
            score_resultado_eliminado = resultado[variable_a_eliminar]
            
            if score_resultado < score_resultado_eliminado:
                soluciones_actual.remove(variable_a_eliminar)
                eliminados.append(variable_a_eliminar)
                score_resultado = score_resultado_eliminado
                k = 0
                        
        if len(columnas) == len(datos.columns)-1:
            k = k+1
        
        if len(añadidos)<len(columnas):
            clave = ', '.join(soluciones_actual)
            diccionario_resultado[clave] = score_resultado

    done = time.time()
    elapsed = done - start
    print("Tiempo empleado: ", elapsed)
    return diccionario_resultado 

In [42]:
resultado_SFFS = SFFS(datos, "diagnosis")

Tiempo empleado:  73.66840600967407


In [43]:
mostrar_datos(resultado_SFFS)

,0,1,2
0,Mejor Solución,Tamaño,Rendimiento
1,worst concave points,1,0.879769
2,"worst concave points, worst radius",2,0.917158
3,"worst concave points, worst radius, mean texture",3,0.939962
4,"worst concave points, worst radius, mean texture, worst smoothness",4,0.947046
5,"worst radius, mean texture, worst smoothness, worst concavity",4,0.953174
6,"worst radius, mean texture, worst smoothness, worst concavity, mean perimeter",5,0.954045
7,"worst radius, mean texture, worst smoothness, worst concavity, mean perimeter, concavity error",6,0.954284
8,"mean texture, worst smoothness, worst concavity, mean perimeter, concavity error, worst area",6,0.955995
